In [18]:
import torch
import torch.nn as nn
from torch import functional as F
from torch.utils.data import IterableDataset
from torch.utils.data import DataLoader
import numpy as np

In [209]:
class data_loader_(IterableDataset):
    def __init__(self, list_csv:list, chunk_size:int = 3, lags = 512, **kwargs):
        super(data_loader_).__init__(**kwargs)
        """
        list_csv : list of csv file 
        chunk_size : chunk_size many csv file will open up into memory and converted into tensors
        """
        self.csv_files = np.array(list_csv)
        self.chunk_size = chunk_size
        self.lags = lags
        ### -- technical stuff -- ###
        self.num_csv = len(self.csv_files)
        self.csv_index = [i for i in range(self.num_csv)]
        ### --- indexing --- ###
        
    def __iter__(self):
        ### shuffle the things before getting started ###
        np.random.shuffle(self.csv_index)
        for indice in range(int((self.num_csv/self.chunk_size))+1):
            indices = self.csv_index[indice*self.chunk_size: self.chunk_size*(indice+1)]
            csv_files = self.csv_files[indices]
            # csv_files_ = [pd.read_csv(csv_file) for csv_file in csv_files]
            # csv_lengths = [len(df)-self.lags-1 for df in csv_files_] ## this includes the sliding window horizon
            # enumerate the involce time series
            # [[1,2,3,4,5,6], 1]
            # [[1,2,3,4,5,6,7], 2]
            # [[1,2,3,4,5,6,7,8,9,10], 3]
            # l_1 = l_1
            # l_2 = l_1 + l_2
            # l_3 = l_1 + l_2 + l_3
            
            
            yield csv_files
        

In [215]:
K = []
for i in list(data_loader_([i**3 for i in range(11)])):
    print(i)

[343   0   8]
[  1 216 729]
[ 512   27 1000]
[ 64 125]


In [76]:
[1,2,3,4,5][4:10]

[5]

In [16]:
class MyIterableDataset(torch.utils.data.IterableDataset):
    def __init__(self, start, end):
        super(MyIterableDataset).__init__()
        assert end > start, "this example code only works with end >= start"
        self.start = start
        self.end = end
    def __iter__(self):
        worker_info = torch.utils.data.get_worker_info()
        if worker_info is None:  # single-process data loading, return the full iterator
            iter_start = self.start
            iter_end = self.end
        else:  # in a worker process
            # split workload
            per_worker = int(math.ceil((self.end - self.start) / float(worker_info.num_workers)))
            worker_id = worker_info.id
            iter_start = self.start + worker_id * per_worker
            iter_end = min(iter_start + per_worker, self.end)
        for i in range(iter_start, iter_end):
            yield i
# should give same set of data as range(3, 7), i.e., [3, 4, 5, 6].
ds = MyIterableDataset(start=0, end=100)

# Single-process loading
print(list(torch.utils.data.DataLoader(ds, num_workers=0, batch_size  = 32)))


[tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]), tensor([32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
        50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63]), tensor([64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81,
        82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95]), tensor([96, 97, 98, 99])]
